Step 4 — Screen Capture + OCR (Option A)

4.2 Capture a screenshot (Python)

In [4]:
import pyautogui

screenshot = pyautogui.screenshot()
screenshot.save("screen.png")

print("Screenshot saved as screen.png")


Screenshot saved as screen.png


Step 5 — OCR

Step 5.1 — Verify OCR setup (ONE cell)

In [6]:
import pytesseract
import PIL.Image as Image

print("OCR ready")


OCR ready


Step 5.2 — Extract text from screenshot

In [8]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [9]:
from PIL import Image
import pytesseract

img = Image.open("screen.png")
text = pytesseract.image_to_string(img)

print("Extracted text:")
print(text)


Extracted text:
J File Edit Selection View Go Run Terminal Help €> Q Search Cty Boag - a x

@ x3 Welcome ‘® screen_capture.ipynb @ eo-
D: > hack > ai_interviewer_local > {@ screen_capture.ipynb > @ import pyautogui
Ql % Generate + Code -++ Markdown | & Runall © Restart % Clear All Outputs | () Jupyter Variables

” B, venv (python 3.12.8)

egnea-u |

2 |> iz

& screenshot = pyautogui-screenshot|()
screenshot. save("screen. png”)

Ce print("Screenshot saved as screen. png”)

Screenshot saved as screen.png

pag Step 4 — Screen Capture + OCR (Option A)

©
%

x @o0A1 Spaces:4 CRLF {} @ Celliof3 O

OomBtD OCONEE

en aeons
Ae Ge gas




Step 6 — UI Analysis Logic

In [10]:
def classify_ui_text(text: str):
    text_lower = text.lower()

    code_keywords = [
        "import ", "def ", "class ", "{", "}", "()", ";",
        "return", "console.log", "#", "//"
    ]

    terminal_keywords = [
        "error", "exception", "traceback", "ps c:\\", "$ ", ">>>",
        "output", "result"
    ]

    slide_keywords = [
        "definition", "advantages", "disadvantages",
        "step", "architecture", "workflow", "introduction"
    ]

    if any(k in text_lower for k in code_keywords):
        return "CODE"

    if any(k in text_lower for k in terminal_keywords):
        return "OUTPUT / TERMINAL"

    if any(k in text_lower for k in slide_keywords):
        return "SLIDES / THEORY"

    return "UNKNOWN"


Step 6.3 — Test It With OCR Output

In [11]:
screen_type = classify_ui_text(text)
print("Detected screen type:", screen_type)


Detected screen type: CODE


Step 7 — Context Builder
It combines everything we captured into a single, structured prompt that an LLM can actually understand and answer correctly.

Step 7.1 — Context Builder Function

In [12]:
def build_llm_context(
    ocr_text: str,
    ui_type: str,
    stt_text: str
):
    """
    Builds a structured context block for the LLM.
    This is deterministic and model-agnostic.
    """

    context = {
        "ui_type": ui_type,
        "screen_text": ocr_text.strip(),
        "interviewer_question": stt_text.strip(),
        "instructions": None
    }

    # UI-aware instruction routing
    if ui_type == "CODE":
        context["instructions"] = (
            "You are seeing source code. "
            "Explain logic, detect bugs, or suggest improvements. "
            "Be concise and accurate."
        )

    elif ui_type == "OUTPUT / TERMINAL":
        context["instructions"] = (
            "You are seeing program output or terminal logs. "
            "Explain errors, outputs, or next debugging steps."
        )

    elif ui_type == "SLIDES / THEORY":
        context["instructions"] = (
            "You are seeing conceptual or theoretical content. "
            "Answer clearly with definitions and examples if helpful."
        )

    else:
        context["instructions"] = (
            "Screen context is unclear. "
            "Answer conservatively and ask for clarification if needed."
        )

    return context


Step 7.2 — Test With Real Data (from your pipeline)

In [13]:
# Example STT text (mock for now)
stt_text = "Can you explain what this code is doing?"

llm_context = build_llm_context(
    ocr_text=text,
    ui_type=screen_type,
    stt_text=stt_text
)

llm_context


{'ui_type': 'CODE',
 'screen_text': 'J File Edit Selection View Go Run Terminal Help €> Q Search Cty Boag - a x\n\n@ x3 Welcome ‘® screen_capture.ipynb @ eo-\nD: > hack > ai_interviewer_local > {@ screen_capture.ipynb > @ import pyautogui\nQl % Generate + Code -++ Markdown | & Runall © Restart % Clear All Outputs | () Jupyter Variables\n\n” B, venv (python 3.12.8)\n\negnea-u |\n\n2 |> iz\n\n& screenshot = pyautogui-screenshot|()\nscreenshot. save("screen. png”)\n\nCe print("Screenshot saved as screen. png”)\n\nScreenshot saved as screen.png\n\npag Step 4 — Screen Capture + OCR (Option A)\n\n©\n%\n\nx @o0A1 Spaces:4 CRLF {} @ Celliof3 O\n\nOomBtD OCONEE\n\nen aeons\nAe Ge gas',
 'interviewer_question': 'Can you explain what this code is doing?',
 'instructions': 'You are seeing source code. Explain logic, detect bugs, or suggest improvements. Be concise and accurate.'}

Step 7.3 — (Optional) Convert to Prompt String

In [14]:
def context_to_prompt(context: dict) -> str:
    return f"""
Screen Type:
{context['ui_type']}

Visible Screen Content:
{context['screen_text']}

Interviewer Question:
{context['interviewer_question']}

Instruction:
{context['instructions']}
""".strip()


In [15]:
prompt = context_to_prompt(llm_context)
print(prompt)


Screen Type:
CODE

Visible Screen Content:
J File Edit Selection View Go Run Terminal Help €> Q Search Cty Boag - a x

@ x3 Welcome ‘® screen_capture.ipynb @ eo-
D: > hack > ai_interviewer_local > {@ screen_capture.ipynb > @ import pyautogui
Ql % Generate + Code -++ Markdown | & Runall © Restart % Clear All Outputs | () Jupyter Variables

” B, venv (python 3.12.8)

egnea-u |

2 |> iz

& screenshot = pyautogui-screenshot|()
screenshot. save("screen. png”)

Ce print("Screenshot saved as screen. png”)

Screenshot saved as screen.png

pag Step 4 — Screen Capture + OCR (Option A)

©
%

x @o0A1 Spaces:4 CRLF {} @ Celliof3 O

OomBtD OCONEE

en aeons
Ae Ge gas

Interviewer Question:
Can you explain what this code is doing?

Instruction:
You are seeing source code. Explain logic, detect bugs, or suggest improvements. Be concise and accurate.


Step 8 — Interview Logic (Prompts & Flow)
This step controls how the LLM behaves across an interview, not just how it answers one question.

8.3 Base System Prompt (Engine-level)

In [16]:
SYSTEM_PROMPT = """
You are an AI interview assistant.

You must:
- Answer as if you are helping a candidate in a live technical interview.
- Be concise, correct, and structured.
- Prefer clarity over verbosity.
- Never hallucinate missing details.
- Ask for clarification when context is insufficient.
"""


8.4 Dynamic Interview Prompt Builder

In [17]:
def build_interview_prompt(llm_context):
    ui_type = llm_context["ui_type"]
    screen_text = llm_context["screen_text"]
    question = llm_context["interview_question"]
    instructions = llm_context["instructions"]

    prompt = f"""
Screen Type: {ui_type}

Visible Screen Content:
{screen_text}

Interview Question:
{question}

Guidelines:
{instructions}
"""

    return prompt.strip()


8.5 Follow-Up Logic (Critical)

In [18]:
def decide_followup(response_text, ui_type):
    response_lower = response_text.lower()

    if ui_type == "CODE":
        if "error" in response_lower or "bug" in response_lower:
            return "ASK_FIX"
        if "complexity" in response_lower:
            return "ASK_OPTIMIZATION"

    if ui_type == "SLIDES":
        if "example" not in response_lower:
            return "ASK_EXAMPLE"

    if ui_type == "OUTPUT":
        if "why" not in response_lower:
            return "ASK_REASONING"

    return "CONTINUE"


8.6 Stop Conditions (Very Important)

In [19]:
def should_stop(stt_text):
    stop_phrases = [
        "okay got it",
        "thank you",
        "move on",
        "next question",
        "that's enough"
    ]

    text = stt_text.lower()
    return any(p in text for p in stop_phrases)


Step 9 — Evaluation & Scoring Rubric Convert the LLM’s natural language answer into structured scores that represent how well a candidate performed — exactly how a real interviewer thinks.

In [20]:
def evaluate_response(response_text, ui_type):
    text = response_text.lower()

    clarity_score = 3
    tech_score = 3

    # --- Clarity heuristics ---
    if len(response_text.split()) < 20:
        clarity_score -= 1

    if any(k in text for k in ["first", "second", "because", "therefore"]):
        clarity_score += 1

    if any(k in text for k in ["i think", "maybe", "not sure"]):
        clarity_score -= 1

    clarity_score = max(1, min(5, clarity_score))

    # --- Technical heuristics ---
    if ui_type == "CODE":
        if any(k in text for k in ["bug", "error", "fix", "optimize", "complexity"]):
            tech_score += 1
        if "syntax" in text and "logic" not in text:
            tech_score -= 1

    if ui_type == "SLIDES":
        if "example" in text:
            tech_score += 1

    if ui_type == "OUTPUT":
        if "why" in text or "because" in text:
            tech_score += 1

    tech_score = max(1, min(5, tech_score))

    return {
        "clarity_score": clarity_score,
        "technical_score": tech_score,
        "overall_score": round((clarity_score + tech_score) / 2, 1)
    }


Step 9.1 — Test with a mock LLM response

In [21]:
# Mock LLM response (simulate interview answer)
llm_response = """
This code captures a screenshot using pyautogui and saves it as screen.png.
It then uses OCR to extract visible text from the screen.
The complexity is minimal since it runs once per capture.
"""

# Use detected UI type from Step 6
ui_type = screen_type  # should be "CODE"

scores = evaluate_response(llm_response, ui_type)
scores


{'clarity_score': 3, 'technical_score': 4, 'overall_score': 3.5}

Step 10 — Feedback & Report Generation This converts scores + interview signals into a human-readable interview report.

10.1 Feedback Generator (Core Function)

In [22]:
def generate_feedback_report(scores, ui_type, question, answer):
    clarity = scores["clarity_score"]
    tech = scores["technical_score"]
    overall = scores["overall_score"]

    strengths = []
    improvements = []

    # ---- Clarity evaluation ----
    if clarity >= 4:
        strengths.append("Explains ideas clearly and in a structured way.")
    elif clarity == 3:
        improvements.append("Improve clarity by structuring the explanation step-by-step.")
    else:
        improvements.append("Explanation lacks clarity and structure.")

    # ---- Technical evaluation ----
    if tech >= 4:
        strengths.append("Demonstrates strong technical understanding.")
    elif tech == 3:
        improvements.append("Technical depth is acceptable but can be improved.")
    else:
        improvements.append("Technical explanation is shallow or partially incorrect.")

    # ---- UI-type specific feedback ----
    if ui_type == "CODE":
        strengths.append("Able to reason about source code.")
        improvements.append("Consider discussing edge cases, complexity, or optimizations.")
    elif ui_type == "SLIDES / THEORY":
        improvements.append("Provide concrete examples to support theoretical concepts.")
    elif ui_type == "OUTPUT / TERMINAL":
        improvements.append("Explain why the output occurs, not just what it shows.")

    # ---- Final summary ----
    if overall >= 4:
        verdict = "Strong performance. Candidate demonstrates interview readiness."
    elif overall >= 3:
        verdict = "Average performance. Candidate shows potential but needs refinement."
    else:
        verdict = "Below expectations. Significant improvement needed."

    return {
        "question": question,
        "ui_type": ui_type,
        "scores": scores,
        "strengths": strengths,
        "improvements": improvements,
        "final_summary": verdict
    }


10.2 Test With Your Existing Data

In [23]:
report = generate_feedback_report(
    scores=scores,
    ui_type=screen_type,
    question=stt_text,
    answer=llm_response
)

report


{'question': 'Can you explain what this code is doing?',
 'ui_type': 'CODE',
 'scores': {'clarity_score': 3, 'technical_score': 4, 'overall_score': 3.5},
 'strengths': ['Demonstrates strong technical understanding.',
  'Able to reason about source code.'],
 'improvements': ['Improve clarity by structuring the explanation step-by-step.',
  'Consider discussing edge cases, complexity, or optimizations.'],
 'final_summary': 'Average performance. Candidate shows potential but needs refinement.'}

Step 11 — User Interface (Demo UI / CLI)

Step 11.2 — Create a CLI Runner Function

In [24]:
def run_interview_demo():
    print("=" * 50)
    print("AI Interviewer — Demo Mode")
    print("=" * 50)

    print("\n[1] Detecting screen context...")
    print("Screen type:", screen_type)

    print("\n[2] Visible screen text (OCR):")
    print(text[:500], "...")

    print("\n[3] Interview question (STT):")
    print(stt_text)

    print("\n[4] AI Answer:")
    print(llm_response)

    print("\n[5] Evaluation Scores:")
    for k, v in scores.items():
        print(f"  {k}: {v}")

    print("\n[6] Feedback Report:")
    print("Strengths:")
    for s in report["strengths"]:
        print(" -", s)

    print("Improvements:")
    for i in report["improvements"]:
        print(" -", i)

    print("\nFinal Verdict:")
    print(report["final_summary"])

    print("\nInterview session complete.")


Step 11.3 — Run the Demo UI

In [25]:
run_interview_demo()


AI Interviewer — Demo Mode

[1] Detecting screen context...
Screen type: CODE

[2] Visible screen text (OCR):
J File Edit Selection View Go Run Terminal Help €> Q Search Cty Boag - a x

@ x3 Welcome ‘® screen_capture.ipynb @ eo-
D: > hack > ai_interviewer_local > {@ screen_capture.ipynb > @ import pyautogui
Ql % Generate + Code -++ Markdown | & Runall © Restart % Clear All Outputs | () Jupyter Variables

” B, venv (python 3.12.8)

egnea-u |

2 |> iz

& screenshot = pyautogui-screenshot|()
screenshot. save("screen. png”)

Ce print("Screenshot saved as screen. png”)

Screenshot saved as screen.png

pag St ...

[3] Interview question (STT):
Can you explain what this code is doing?

[4] AI Answer:

This code captures a screenshot using pyautogui and saves it as screen.png.
It then uses OCR to extract visible text from the screen.
The complexity is minimal since it runs once per capture.


[5] Evaluation Scores:
  clarity_score: 3
  technical_score: 4
  overall_score: 3.5

[6] Feedback Repo